In [3]:
import cv2
import numpy as np

# Open webcam
vidBuffer = cv2.VideoCapture(0)
framerate = 60
delayFrame = int(1.0 / framerate * 1000)

while True:
    flag, img = vidBuffer.read()
    if not flag:
        break

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Use Otsu threshold for automatic separation
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Morphological operations to clean noise & fill holes
    kernel = np.ones((5, 5), np.uint8)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Build mask
    mask = np.zeros(img.shape[:2], np.uint8)
    for cnt in contours:
        if cv2.contourArea(cnt) > 5000:  # ignore tiny blobs
            cv2.drawContours(mask, [cnt], -1, 255, thickness=-1)

    mask = cv2.bitwise_not(mask)  # Invert mask


    # Invert mask
    mask_inv = cv2.bitwise_not(mask)

    # Extract person
    fg = cv2.bitwise_and(img, img, mask=mask)
    
    cv2.imshow("Person", fg)

    key = cv2.waitKey(delayFrame)
    if key == 27:  # ESC to exit
        break

vidBuffer.release()
cv2.destroyAllWindows()
